In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
dbutils.widgets.text("catalogo", "catalog_proy_fin_dev")
dbutils.widgets.text("esquema_source", "silver")
dbutils.widgets.text("esquema_sink", "golden")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_sink = dbutils.widgets.get("esquema_sink")

In [0]:
df_payroll_transformed = spark.table(f"{catalogo}.{esquema_source}.payroll_transformed")

In [0]:
df_payroll_transformed.limit(5).display()

In [0]:
df_transformed = df_payroll_transformed.groupBy(col("pay_year")).agg(
                                                     count(col("location_desc")).alias("conteo"),
                                                     max(col("income_tax_withheld")).alias("max_tax_withheld"),
                                                     min(col("employee_benefits")).alias("min_benefits"),
                                                     max(col("hourly_rate")).alias("hourly_rate"),
                                                     max(col("Location")).alias("Local_type")
                                                     ).orderBy(col("pay_year").desc())

In [0]:
df_transformed.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_sink}.golden_payroll_partitioned")